# Nuisance Alarm Filter

## Documentation

Object is to join anomaly detection results with the rest of the event attributes

## Initialisation

### Load libraries

In [1]:
# Load libraries
import pandas as pd
import numpy as np
import datetime as dt
import time

### Configure display

In [2]:
# Enable display of all columns for dataframes with many variables
pd.set_option('display.max_columns', None)

### Set Up Directory

In [3]:
# Check current directory location
import os
cwd = os.getcwd()
cwd

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\scripts'

In [4]:
# Define root file directory folder where the files are being stored
#os.chdir(cwd + alarmLoc)
os.chdir(os.path.dirname(os.getcwd()) + '\\alarm-event-logs')

# Check current directory location
cwd = os.getcwd()

# Check directory location
cwd

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs'

In [5]:
# Choose to whether to process logs from Engineering Hours (EngHours) or not
# Set to "True" to process engineering hour logs
# Set to "False" to process non-engineering hour logs
EngHours = True
if (EngHours == True):
    EngHrTag = "-EngHr"
elif (EngHours == False):
    EngHrTag = ""

# Define type of files from which server (ats/cms/ecs) to process
targetFiles  = "cms"

# Location of Alarm and Normal Event Files
if (targetFiles == "ats") and (EngHours == False):
    eventAttr = '\\taggedOutput\\AnomalyTaggingRaw\\run 05\\ats'
    anomalyTag = '\\taggedOutput\\AnomalyTaggingResults\\run 05\\ats'
    saveLoc = '\\taggedOutput\\AnomalyTaggingResultsFULL\\run 05\\ats'
    print(eventAttr)
    print(anomalyTag)
elif (targetFiles == "ats") and (EngHours == True):
    eventAttr = '\\taggedOutput\\AnomalyTaggingRaw\\run 05\\ats-eng'
    anomalyTag = '\\taggedOutput\\AnomalyTaggingResults\\run 05\\ats-eng'
    saveLoc = '\\taggedOutput\\AnomalyTaggingResultsFULL\\run 05\\ats'
    print(eventAttr)
    print(anomalyTag)
elif (targetFiles == "cms") and (EngHours == False):
    eventAttr = '\\taggedOutput\\AnomalyTaggingRaw\\run 05\\cms'
    anomalyTag = '\\taggedOutput\\AnomalyTaggingResults\\run 05\\cms'
    saveLoc = '\\taggedOutput\\AnomalyTaggingResultsFULL\\run 05\\cms'
    print(eventAttr)
    print(anomalyTag)
elif (targetFiles == "cms") and (EngHours == True):
    eventAttr = '\\taggedOutput\\AnomalyTaggingRaw\\run 05\\cms-eng'
    anomalyTag = '\\taggedOutput\\AnomalyTaggingResults\\run 05\\cms-eng'
    saveLoc = '\\taggedOutput\\AnomalyTaggingResultsFULL\\run 05\\cms'
    print(eventAttr)
    print(anomalyTag)
elif (targetFiles == "ecs") and (EngHours == False):
    eventAttr = '\\taggedOutput\\AnomalyTaggingRaw\\run 05\\ecs'
    anomalyTag = '\\taggedOutput\\AnomalyTaggingResults\\run 05\\ecs'
    saveLoc = '\\taggedOutput\\AnomalyTaggingResultsFULL\\run 05\\ecs'
    print(eventAttr)
    print(anomalyTag)
elif (targetFiles == "ecs") and (EngHours == True):
    eventAttr = '\\taggedOutput\\AnomalyTaggingRaw\\run 05\\ecs-eng'
    anomalyTag = '\\taggedOutput\\AnomalyTaggingResults\\run 05\\ecs-eng'
    saveLoc = '\\taggedOutput\\AnomalyTaggingResultsFULL\\run 05\\ecs'
    print(eventAttr)
    print(anomalyTag)
else:
    print("Error, choose another file)")

\taggedOutput\AnomalyTaggingRaw\run 05\cms-eng
\taggedOutput\AnomalyTaggingResults\run 05\cms-eng


## Anomaly Tagging

### Load Files with Anomaly Tags

In [6]:
# Define root file directory folder where the files are being stored
os.chdir(cwd + anomalyTag)
#os.chdir(cwd + eventLoc)
# Check directory location
os.getcwd()

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs\\taggedOutput\\AnomalyTaggingResults\\run 05\\cms-eng'

In [7]:
# Print out list of relevant files
for f in os.listdir(os.getcwd()):
    if f.endswith('csv'):
        print(f)

CMS-05-AnomalyTagging_Output-EngHr-AltRun.csv


In [8]:
# Gather list of cleaned alarmList files
dfs = [pd.read_csv(f)
        for f in os.listdir(os.getcwd()) if f.endswith('csv')]

# Compile list of cleaned alarmList files into a single dataframe
df_anomaly = pd.concat(dfs, ignore_index=True).reset_index()

# Delete redundant index col
del df_anomaly["index"]

# Print df summary stats
print(df_anomaly.shape)
print(df_anomaly.info())

(312160, 24)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312160 entries, 0 to 312159
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   EQUIPMENT_NAME         312160 non-null  float64
 1   ASSET_ID_RAW           312160 non-null  object 
 2   SCS_TIME               312160 non-null  object 
 3   ENVIRONMENT            312160 non-null  object 
 4   GeoCode                312160 non-null  object 
 5   FUNCTIONAL_CATEGORY    312160 non-null  int64  
 6   AssetClass             312160 non-null  object 
 7   AssetSubClass          307757 non-null  object 
 8   EVENT_DESC_CAT         312160 non-null  object 
 9   EVENT_STATUS           312157 non-null  object 
 10  Severity_Class         312160 non-null  object 
 11  Outlier_PCA            312160 non-null  bool   
 12  PCA_AnomalyProb        312160 non-null  float64
 13  Outlier_IsoForest      312160 non-null  bool   
 14  IsoForest_AnomalyProb  

In [9]:
# Inspect df
df_anomaly.head()

,EQUIPMENT_NAME,ASSET_ID_RAW,SCS_TIME,ENVIRONMENT,GeoCode,FUNCTIONAL_CATEGORY,AssetClass,AssetSubClass,EVENT_DESC_CAT,EVENT_STATUS,Severity_Class,Outlier_PCA,PCA_AnomalyProb,Outlier_IsoForest,IsoForest_AnomalyProb,Outlier_HBOS,HBOS_AnomalyProb,Outlier_CBLOF,CBLOF_AnomalyProb,Outlier_LODA,LODA_AnomalyProb,AnomalyProb,AnomalyProbClass,Outlier_Strength
0,0.0,TRACTION_BGK_OFF,2021-01-01 01:48:02.819180032,OCCCMS,OCC,1,TRACTION,TRACTION,"DM, DM, DI, DFS, DFN, DFN, DFS - Open Control",REQUESTED,Operational-Low,False,0.316497,True,0.643964,False,0.837822,False,0.772957,False,0.746246,0.663497,False,1
1,0.0,TRACTION_BGK_OFF,2021-01-01 01:50:14.752684032,OCCCMS,OCC,1,TRACTION,TRACTION,"DM, DM, DI, DFS, DFN, DFN, DFS - Open Control",TERMINATED,Operational-Low,False,0.309214,False,0.362036,False,0.776042,False,0.520069,False,0.031734,0.399819,False,0
2,0.0,TRACTION_BGK_OFF,2021-01-02 00:37:56.958477056,OCCCMS,OCC,1,TRACTION,TRACTION,"DM, DM, DI, DFS, DFN, DFN, DFS - Open Control",REQUESTED,Operational-Low,False,0.318976,False,0.425982,False,0.776042,False,0.550849,False,0.031734,0.420717,False,0
3,0.0,TRACTION_BGK_OFF,2021-01-02 00:40:23.753911040,OCCCMS,OCC,1,TRACTION,TRACTION,"DM, DM, DI, DFS, DFN, DFN, DFS - Open Control",TERMINATED,Operational-Low,False,0.311145,False,0.359097,False,0.776042,False,0.534248,False,0.031734,0.402453,False,0
4,0.0,TRACTION_BGK_OFF,2021-01-03 00:52:01.285080064,OCCCMS,OCC,1,TRACTION,TRACTION,"DM, DM, DI, DFS, DFN, DFN, DFS - Open Control",REQUESTED,Operational-Low,False,0.314038,False,0.364794,False,0.776042,False,0.533225,False,0.031734,0.403967,False,0


In [10]:
# Inspect df
df_anomaly.tail()

,EQUIPMENT_NAME,ASSET_ID_RAW,SCS_TIME,ENVIRONMENT,GeoCode,FUNCTIONAL_CATEGORY,AssetClass,AssetSubClass,EVENT_DESC_CAT,EVENT_STATUS,Severity_Class,Outlier_PCA,PCA_AnomalyProb,Outlier_IsoForest,IsoForest_AnomalyProb,Outlier_HBOS,HBOS_AnomalyProb,Outlier_CBLOF,CBLOF_AnomalyProb,Outlier_LODA,LODA_AnomalyProb,AnomalyProb,AnomalyProbClass,Outlier_Strength
312155,0.0,SCS/NED/1212/GWS04,2021-01-03 01:46:21.702497024,OCCCMS,OCC,74,SCS,GWS,Operator Logged In/Out of NelVisu,SUCCEEDED,Operational-Low,False,0.244243,False,0.280767,False,0.849186,False,0.147996,False,0.473629,0.399164,False,0
312156,0.0,SCS/NED/1212/GWS04,2021-01-11 03:25:26.227955200,OCCCMS,OCC,74,SCS,GWS,Operator Logged In/Out of NelVisu,SUCCEEDED,Operational-Low,False,0.249256,False,0.320808,False,0.849186,False,0.150845,False,0.473629,0.408745,False,0
312157,0.0,SCS/NED/1212/GWS04,2021-01-11 03:25:26.734913024,OCCCMS,OCC,74,SCS,GWS,Operator Logged In/Out of NelVisu,SUCCEEDED,Operational-Low,False,0.244247,False,0.265074,False,0.869366,False,0.121199,False,0.774272,0.454831,False,0
312158,0.0,SCS/NED/1212/GWS04,2021-01-11 03:25:27.347742976,OCCCMS,OCC,74,SCS,GWS,Operator Logged In/Out of NelVisu,SUCCEEDED,Operational-Low,False,0.244183,False,0.265301,False,0.869366,False,0.121003,False,0.774272,0.454825,False,0
312159,0.0,SCS/NED/1212/GWS04,2021-01-13 01:19:49.153079040,OCCCMS,OCC,74,SCS,GWS,Operator Logged In/Out of NelVisu,SUCCEEDED,Operational-Low,False,0.251185,False,0.364848,False,0.849186,False,0.161100,False,0.473629,0.419990,False,0


### Load Files with Full Event Attribute Data

In [11]:
# Check directory location
cwd

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs'

In [12]:
# Define root file directory folder where the files are being stored
#os.chdir(cwd + alarmLoc)
os.chdir(cwd + eventAttr)
# Check directory location
os.getcwd()

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs\\taggedOutput\\AnomalyTaggingRaw\\run 05\\cms-eng'

In [13]:
# Print out list of relevant files
# Make sure that the files are in the same order as the list of files with the anomaly tags
for f in os.listdir(os.getcwd()):
    if f.endswith('csv'):
        print(f)

CMS-05-AnomalyTagging_RAW-EngHr-AltRun.csv


In [14]:
# Gather list of cleaned alarmList files
dfs = [pd.read_csv(f)
        for f in os.listdir(os.getcwd()) if f.endswith('csv')]

# Compile list of cleaned alarmList files into a single dataframe
df_eventAttr = pd.concat(dfs, ignore_index=True).reset_index()

# Delete redundant index col
del df_eventAttr["index"]

# Print df summary stats
print(df_eventAttr.shape)
print(df_eventAttr.info())

(312160, 52)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312160 entries, 0 to 312159
Data columns (total 52 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   EQUIPMENT_NAME                    312160 non-null  float64
 1   SCS_TIME                          312160 non-null  object 
 2   FUNCTIONAL_CATEGORY               312160 non-null  int64  
 3   ENVIRONMENT                       312160 non-null  object 
 4   ASSET_ID_RAW                      312160 non-null  object 
 5   EVENT_STATUS                      312157 non-null  object 
 6   ASSET_DESC_CAT                    311610 non-null  object 
 7   EVENT_DESC_CAT                    312160 non-null  object 
 8   AssetClass                        312160 non-null  object 
 9   AssetSubClass                     307757 non-null  object 
 10  isAlarm                           312160 non-null  bool   
 11  NuisanceAlarm                     31216

In [15]:
# Inspect df
df_eventAttr.head()

,EQUIPMENT_NAME,SCS_TIME,FUNCTIONAL_CATEGORY,ENVIRONMENT,ASSET_ID_RAW,EVENT_STATUS,ASSET_DESC_CAT,EVENT_DESC_CAT,AssetClass,AssetSubClass,isAlarm,NuisanceAlarm,RepeatAlarm,AltAlarm2,AltAlarm3,GeoCode,GeoSector,Severity_Class,SeverityRank,CrashWarning,DayofWeek,Weekend,HourofDay,EngHours,RWEC_Ratio_Flat,RWEC_Ratio_Seasonal,RWEC_Ratio_SeasonalAlarm,RWSS_LocAsset,RWSS_LocAssetClass,RWSS_Loc,EventAttr,sentimentScore,SentimentClass,SEVERITY_N-1,SEVERITY_Worsen,isAlarm_N-1,NuisanceAlarm_N-1,CrashWarning_N-1,sentimentScore_N-1,SentimentClass_N-1,sentimentScoreDelta,SentimentClassImprove_N-1,SCS_TIME_logDelta_N-1,SEVERITY_GEO_N-1,isAlarm_GEO_N-1,NuisanceAlarm_GEO_N-1,CrashWarning_GEO_N-1,sentimentScore_GEO_N-1,SCS_TIME_logDelta_GEO_N-1,RWEC_Ratio_Flat_GEO_N-1,RWEC_Ratio_Seasonal_GEO_N-1,RWEC_Ratio_SeasonalAlarm_GEO_N-1
0,0.0,2021-01-01 01:48:02.819180032,1,OCCCMS,TRACTION_BGK_OFF,REQUESTED,NaN,"DM, DM, DI, DFS, DFN, DFN, DFS - Open Control",TRACTION,TRACTION,False,True,False,True,True,OCC,10,Operational-Low,3,False,4,False,1,True,0.0,0.0,0.0,0.3,0.3,1.215217,"1 - DM, DM, DI, DFS, DFN, DFN, DFS - Open Cont...",0.0,0,0,True,False,False,False,0.0,0,0.0,False,0.0,0,False,False,False,0.0,0.000000,0.0,0.0,0.0
1,0.0,2021-01-01 01:50:14.752684032,1,OCCCMS,TRACTION_BGK_OFF,TERMINATED,NaN,"DM, DM, DI, DFS, DFN, DFN, DFS - Open Control",TRACTION,TRACTION,False,True,False,True,True,OCC,10,Operational-Low,3,False,4,False,1,True,0.0,0.0,0.0,0.3,0.3,0.994118,"1 - DM, DM, DI, DFS, DFN, DFN, DFS - Open Cont...",0.0,0,0,True,False,False,False,0.0,0,0.0,False,0.0,3,False,True,False,0.0,2.117271,0.0,0.0,0.0
2,0.0,2021-01-02 00:37:56.958477056,1,OCCCMS,TRACTION_BGK_OFF,REQUESTED,NaN,"DM, DM, DI, DFS, DFN, DFN, DFS - Open Control",TRACTION,TRACTION,False,True,False,True,True,OCC,10,Operational-Low,3,False,5,True,0,True,0.0,0.0,0.0,0.3,0.3,0.313333,"1 - DM, DM, DI, DFS, DFN, DFN, DFS - Open Cont...",0.0,0,0,True,False,False,False,0.0,0,0.0,False,0.0,3,False,True,False,0.0,4.914142,0.0,0.0,0.0
3,0.0,2021-01-02 00:40:23.753911040,1,OCCCMS,TRACTION_BGK_OFF,TERMINATED,NaN,"DM, DM, DI, DFS, DFN, DFN, DFS - Open Control",TRACTION,TRACTION,False,True,False,True,True,OCC,10,Operational-Low,3,False,5,True,0,True,0.0,0.0,0.0,0.3,0.3,0.310000,"1 - DM, DM, DI, DFS, DFN, DFN, DFS - Open Cont...",0.0,0,0,True,False,False,False,0.0,0,0.0,False,0.0,3,False,True,False,0.0,2.164353,0.0,0.0,0.0
4,0.0,2021-01-03 00:52:01.285080064,1,OCCCMS,TRACTION_BGK_OFF,REQUESTED,NaN,"DM, DM, DI, DFS, DFN, DFN, DFS - Open Control",TRACTION,TRACTION,False,True,False,True,True,OCC,10,Operational-Low,3,False,6,True,0,True,0.0,0.0,0.0,0.3,0.3,0.263636,"1 - DM, DM, DI, DFS, DFN, DFN, DFS - Open Cont...",0.0,0,0,True,False,False,False,0.0,0,0.0,False,0.0,3,False,True,False,0.0,2.843233,0.0,0.0,0.0


In [16]:
# Inspect df
df_eventAttr.tail()

,EQUIPMENT_NAME,SCS_TIME,FUNCTIONAL_CATEGORY,ENVIRONMENT,ASSET_ID_RAW,EVENT_STATUS,ASSET_DESC_CAT,EVENT_DESC_CAT,AssetClass,AssetSubClass,isAlarm,NuisanceAlarm,RepeatAlarm,AltAlarm2,AltAlarm3,GeoCode,GeoSector,Severity_Class,SeverityRank,CrashWarning,DayofWeek,Weekend,HourofDay,EngHours,RWEC_Ratio_Flat,RWEC_Ratio_Seasonal,RWEC_Ratio_SeasonalAlarm,RWSS_LocAsset,RWSS_LocAssetClass,RWSS_Loc,EventAttr,sentimentScore,SentimentClass,SEVERITY_N-1,SEVERITY_Worsen,isAlarm_N-1,NuisanceAlarm_N-1,CrashWarning_N-1,sentimentScore_N-1,SentimentClass_N-1,sentimentScoreDelta,SentimentClassImprove_N-1,SCS_TIME_logDelta_N-1,SEVERITY_GEO_N-1,isAlarm_GEO_N-1,NuisanceAlarm_GEO_N-1,CrashWarning_GEO_N-1,sentimentScore_GEO_N-1,SCS_TIME_logDelta_GEO_N-1,RWEC_Ratio_Flat_GEO_N-1,RWEC_Ratio_Seasonal_GEO_N-1,RWEC_Ratio_SeasonalAlarm_GEO_N-1
312155,0.0,2021-01-03 01:46:21.702497024,74,OCCCMS,SCS/NED/1212/GWS04,SUCCEEDED,NelVisu,Operator Logged In/Out of NelVisu,SCS,GWS,False,False,False,False,False,OCC,10,Operational-Low,3,False,6,True,1,True,0.0,0.0,0.0,1.35,1.582143,1.354000,74 - Operator Logged In/Out of NelVisu - SUCCE...,0.0,0,0,True,False,False,False,0.0,0,0.0,False,0.0,3,False,False,False,0.0,0.602060,0.0,0.0,0.0
312156,0.0,2021-01-11 03:25:26.227955200,74,OCCCMS,SCS/NED/1212/GWS04,SUCCEEDED,NelVisu,Operator Logged In/Out of NelVisu,SCS,GWS,False,False,False,False,False,OCC,10,Operational-Low,3,False,0,False,3,True,0.0,0.0,0.0,1.35,1.905882,1.403571,74 - Operator Logged In/Out of NelVisu - SUCCE...,0.0,0,0,True,False,False,False,0.0,0,0.0,False,0.0,3,False,False,False,0.0,3.774079,0.0,0.0,0.0
312157,0.0,2021-01-11 03:25:26.734913024,74,OCCCMS,SCS/NED/1212/GWS04,SUCCEEDED,NelVisu,Operator Logged In/Out of NelVisu,SCS,GWS,False,False,False,False,False,OCC,10,Operational-Low,3,False,0,False,3,True,0.0,0.0,0.0,1.35,1.875000,1.401724,74 - Operator Logged In/Out of NelVisu - SUCCE...,0.0,0,0,True,False,False,False,0.0,0,0.0,False,0.0,3,False,False,False,0.0,0.000000,0.0,0.0,0.0
312158,0.0,2021-01-11 03:25:27.347742976,74,OCCCMS,SCS/NED/1212/GWS04,SUCCEEDED,NelVisu,Operator Logged In/Out of NelVisu,SCS,GWS,False,False,False,False,False,OCC,10,Operational-Low,3,False,0,False,3,True,0.0,0.0,0.0,1.35,1.847368,1.400000,74 - Operator Logged In/Out of NelVisu - SUCCE...,0.0,0,0,True,False,False,False,0.0,0,0.0,False,0.0,3,False,False,False,0.0,0.000000,0.0,0.0,0.0
312159,0.0,2021-01-13 01:19:49.153079040,74,OCCCMS,SCS/NED/1212/GWS04,SUCCEEDED,NelVisu,Operator Logged In/Out of NelVisu,SCS,GWS,False,False,False,False,False,OCC,10,Operational-Low,3,False,2,False,1,True,0.0,0.0,0.0,1.35,1.350000,1.350000,74 - Operator Logged In/Out of NelVisu - SUCCE...,0.0,0,0,True,False,False,False,0.0,0,0.0,False,0.0,3,False,False,False,0.0,4.896862,0.0,0.0,0.0


#### Quick Export of Unique List of Event Attributes (optional step for ease of diagnostics)

In [17]:
# Define Save Location
saveLoc = saveLoc
#os.chdir(cwd + alarmLoc)
os.chdir(cwd + saveLoc)
# Check directory location
print(os.getcwd())

C:\Users\cftfda01\Documents\SBST Train IAMS Project\alarm-event-logs\taggedOutput\AnomalyTaggingResultsFULL\run 05\cms


In [18]:
# Export Data
quickExport = False
if (quickExport == True):
    # Filter out only for unique Event Attribute values
    eventAttrSeries = pd.Series(df_eventAttr["EventAttr"].unique())
    
    # Define File Save Parameters
    FileName = targetFiles
    Run = "-B0001"
    # True for single file output; # False for multiple file output; "both" for both Single & Multiple File Output
    singleSave = "both"
    # Define source data
    df = eventAttrSeries

    # Get length of dataframe
    df_len = df.shape[0]
    # Inspect data
    print(df_len)

    # Define Size of Partitioned Dataframes
    partionSize = 500000

    # Define Number of Partitions (Always Round Up to Nearest Interger)
    if (df_len == partionSize): 
        partitions = 1
    else:
        partitions = df_len // partionSize + 1

    # Inspect data
    print(partitions)
    
    # Export file based on above settings
    if singleSave == True:
        fileNameN = FileName + Run + '-EventAttrUnique' + '.csv'
        df.to_csv("main UniqueEventAttr/" + fileNameN, index=False)
        print(fileNameN + " SAVED")
    elif singleSave == "both":
        # Single File Save
        fileNameN = FileName + Run + '-EventAttrUnique' + '.csv'
        df.to_csv("main UniqueEventAttr/" + fileNameN, index=False)
        print(fileNameN + " SAVED")

        # Split Dataframe into batches of 500K rows
        for counter in range(partitions):
            startPoint = counter * partionSize
            df_subset = df.iloc[startPoint : (startPoint + partionSize)]
            #print(df_subset.info())

            # Save File
            fileNameN = "Subset File Ver UniqueEventAttr/" + FileName + Run + '-' + str(counter).zfill(3) + '-EventAttrUnique' + '.csv'
            df_subset.to_csv(fileNameN, index=False)
            print(fileNameN + " SAVED")
    else:
        # Split Dataframe into batches of 500K rows
        for counter in range(partitions):
            startPoint = counter * partionSize
            df_subset = df.iloc[startPoint : (startPoint + partionSize)]
            #print(df_subset.info())

            # Save File
            fileNameN = "Subset File Ver UniqueEventAttr/" + FileName + Run + '-' + str(counter).zfill(3) + '-EventAttrUnique' + '.csv'
            df_subset.to_csv(fileNameN, index=False)
            print(fileNameN + " SAVED")

    # Ring Beeper When Complete
    import winsound
    import time

    duration1 = 400  # milliseconds
    freq1 = 400  # Hz
    duration2 = 600  # milliseconds
    freq2 = 300  # Hz
    repeatCount = 7
    for n in range(repeatCount):
        winsound.Beep(freq1, duration1)
        winsound.Beep(freq2, duration2)
        time.sleep(1)
    
    
    del quickExport, eventAttrSeries, df, df_subset
    
else:
    del quickExport


#### Include Filter For Normal Events
Deprecated for it to be included in the BI layer instead for ease of updates to the dictionaries.

In [19]:
# Check directory location
#cwd

In [20]:
# Check directory location
#os.getcwd()

In [21]:
# Location of lookup table
#if (targetFiles == "ats"):
#    lookupTableLoc = '../../../PositiveNeutralEventIndex/ats-B0001-EventAttrUnique.xlsx'
#elif (targetFiles == "cms") :
#    lookupTableLoc = '../../../PositiveNeutralEventIndex/cms-B0001-EventAttrUnique.xlsx'
#else: # (targetFile == "ecs") & catch all
#    lookupTableLoc = '../../../PositiveNeutralEventIndex/ecs-B0001-EventAttrUnique.xlsx'
    
#normalState_df = pd.read_excel(lookupTableLoc)
#normalState_df.head()

In [22]:
# Inspect data
df_eventAttr.head()

,EQUIPMENT_NAME,SCS_TIME,FUNCTIONAL_CATEGORY,ENVIRONMENT,ASSET_ID_RAW,EVENT_STATUS,ASSET_DESC_CAT,EVENT_DESC_CAT,AssetClass,AssetSubClass,isAlarm,NuisanceAlarm,RepeatAlarm,AltAlarm2,AltAlarm3,GeoCode,GeoSector,Severity_Class,SeverityRank,CrashWarning,DayofWeek,Weekend,HourofDay,EngHours,RWEC_Ratio_Flat,RWEC_Ratio_Seasonal,RWEC_Ratio_SeasonalAlarm,RWSS_LocAsset,RWSS_LocAssetClass,RWSS_Loc,EventAttr,sentimentScore,SentimentClass,SEVERITY_N-1,SEVERITY_Worsen,isAlarm_N-1,NuisanceAlarm_N-1,CrashWarning_N-1,sentimentScore_N-1,SentimentClass_N-1,sentimentScoreDelta,SentimentClassImprove_N-1,SCS_TIME_logDelta_N-1,SEVERITY_GEO_N-1,isAlarm_GEO_N-1,NuisanceAlarm_GEO_N-1,CrashWarning_GEO_N-1,sentimentScore_GEO_N-1,SCS_TIME_logDelta_GEO_N-1,RWEC_Ratio_Flat_GEO_N-1,RWEC_Ratio_Seasonal_GEO_N-1,RWEC_Ratio_SeasonalAlarm_GEO_N-1
0,0.0,2021-01-01 01:48:02.819180032,1,OCCCMS,TRACTION_BGK_OFF,REQUESTED,NaN,"DM, DM, DI, DFS, DFN, DFN, DFS - Open Control",TRACTION,TRACTION,False,True,False,True,True,OCC,10,Operational-Low,3,False,4,False,1,True,0.0,0.0,0.0,0.3,0.3,1.215217,"1 - DM, DM, DI, DFS, DFN, DFN, DFS - Open Cont...",0.0,0,0,True,False,False,False,0.0,0,0.0,False,0.0,0,False,False,False,0.0,0.000000,0.0,0.0,0.0
1,0.0,2021-01-01 01:50:14.752684032,1,OCCCMS,TRACTION_BGK_OFF,TERMINATED,NaN,"DM, DM, DI, DFS, DFN, DFN, DFS - Open Control",TRACTION,TRACTION,False,True,False,True,True,OCC,10,Operational-Low,3,False,4,False,1,True,0.0,0.0,0.0,0.3,0.3,0.994118,"1 - DM, DM, DI, DFS, DFN, DFN, DFS - Open Cont...",0.0,0,0,True,False,False,False,0.0,0,0.0,False,0.0,3,False,True,False,0.0,2.117271,0.0,0.0,0.0
2,0.0,2021-01-02 00:37:56.958477056,1,OCCCMS,TRACTION_BGK_OFF,REQUESTED,NaN,"DM, DM, DI, DFS, DFN, DFN, DFS - Open Control",TRACTION,TRACTION,False,True,False,True,True,OCC,10,Operational-Low,3,False,5,True,0,True,0.0,0.0,0.0,0.3,0.3,0.313333,"1 - DM, DM, DI, DFS, DFN, DFN, DFS - Open Cont...",0.0,0,0,True,False,False,False,0.0,0,0.0,False,0.0,3,False,True,False,0.0,4.914142,0.0,0.0,0.0
3,0.0,2021-01-02 00:40:23.753911040,1,OCCCMS,TRACTION_BGK_OFF,TERMINATED,NaN,"DM, DM, DI, DFS, DFN, DFN, DFS - Open Control",TRACTION,TRACTION,False,True,False,True,True,OCC,10,Operational-Low,3,False,5,True,0,True,0.0,0.0,0.0,0.3,0.3,0.310000,"1 - DM, DM, DI, DFS, DFN, DFN, DFS - Open Cont...",0.0,0,0,True,False,False,False,0.0,0,0.0,False,0.0,3,False,True,False,0.0,2.164353,0.0,0.0,0.0
4,0.0,2021-01-03 00:52:01.285080064,1,OCCCMS,TRACTION_BGK_OFF,REQUESTED,NaN,"DM, DM, DI, DFS, DFN, DFN, DFS - Open Control",TRACTION,TRACTION,False,True,False,True,True,OCC,10,Operational-Low,3,False,6,True,0,True,0.0,0.0,0.0,0.3,0.3,0.263636,"1 - DM, DM, DI, DFS, DFN, DFN, DFS - Open Cont...",0.0,0,0,True,False,False,False,0.0,0,0.0,False,0.0,3,False,True,False,0.0,2.843233,0.0,0.0,0.0


In [23]:
# Merge Filter tag data
#df_eventAttr = df_eventAttr.merge(normalState_df, "left", left_on = "EventAttr", right_on = "Event Attribute")

# Inspect data
#df_eventAttr.head()

In [24]:
# Drop redundant fields
#del df_eventAttr["Event Attribute"]

# Rename Column
#df_eventAttr.rename(columns = {'To Ignore (T/F)':'Benign Event To Ignore'}, inplace = True)

# Inspect data
#df_eventAttr.head()

### Append Alarm Tag Info
The sorting order of both sets of files are exactly the same, so directly concantenating both sets of data by index would suffice

In [25]:
# Append in Anomaly Tags
df = pd.concat([df_eventAttr, df_anomaly], axis = 1)

# Inspect data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312160 entries, 0 to 312159
Data columns (total 76 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   EQUIPMENT_NAME                    312160 non-null  float64
 1   SCS_TIME                          312160 non-null  object 
 2   FUNCTIONAL_CATEGORY               312160 non-null  int64  
 3   ENVIRONMENT                       312160 non-null  object 
 4   ASSET_ID_RAW                      312160 non-null  object 
 5   EVENT_STATUS                      312157 non-null  object 
 6   ASSET_DESC_CAT                    311610 non-null  object 
 7   EVENT_DESC_CAT                    312160 non-null  object 
 8   AssetClass                        312160 non-null  object 
 9   AssetSubClass                     307757 non-null  object 
 10  isAlarm                           312160 non-null  bool   
 11  NuisanceAlarm                     312160 non-null  b

In [26]:
# Inspect data
df.head()

,EQUIPMENT_NAME,SCS_TIME,FUNCTIONAL_CATEGORY,ENVIRONMENT,ASSET_ID_RAW,EVENT_STATUS,ASSET_DESC_CAT,EVENT_DESC_CAT,AssetClass,AssetSubClass,isAlarm,NuisanceAlarm,RepeatAlarm,AltAlarm2,AltAlarm3,GeoCode,GeoSector,Severity_Class,SeverityRank,CrashWarning,DayofWeek,Weekend,HourofDay,EngHours,RWEC_Ratio_Flat,RWEC_Ratio_Seasonal,RWEC_Ratio_SeasonalAlarm,RWSS_LocAsset,RWSS_LocAssetClass,RWSS_Loc,EventAttr,sentimentScore,SentimentClass,SEVERITY_N-1,SEVERITY_Worsen,isAlarm_N-1,NuisanceAlarm_N-1,CrashWarning_N-1,sentimentScore_N-1,SentimentClass_N-1,sentimentScoreDelta,SentimentClassImprove_N-1,SCS_TIME_logDelta_N-1,SEVERITY_GEO_N-1,isAlarm_GEO_N-1,NuisanceAlarm_GEO_N-1,CrashWarning_GEO_N-1,sentimentScore_GEO_N-1,SCS_TIME_logDelta_GEO_N-1,RWEC_Ratio_Flat_GEO_N-1,RWEC_Ratio_Seasonal_GEO_N-1,RWEC_Ratio_SeasonalAlarm_GEO_N-1,EQUIPMENT_NAME,ASSET_ID_RAW,SCS_TIME,ENVIRONMENT,GeoCode,FUNCTIONAL_CATEGORY,AssetClass,AssetSubClass,EVENT_DESC_CAT,EVENT_STATUS,Severity_Class,Outlier_PCA,PCA_AnomalyProb,Outlier_IsoForest,IsoForest_AnomalyProb,Outlier_HBOS,HBOS_AnomalyProb,Outlier_CBLOF,CBLOF_AnomalyProb,Outlier_LODA,LODA_AnomalyProb,AnomalyProb,AnomalyProbClass,Outlier_Strength
0,0.0,2021-01-01 01:48:02.819180032,1,OCCCMS,TRACTION_BGK_OFF,REQUESTED,NaN,"DM, DM, DI, DFS, DFN, DFN, DFS - Open Control",TRACTION,TRACTION,False,True,False,True,True,OCC,10,Operational-Low,3,False,4,False,1,True,0.0,0.0,0.0,0.3,0.3,1.215217,"1 - DM, DM, DI, DFS, DFN, DFN, DFS - Open Cont...",0.0,0,0,True,False,False,False,0.0,0,0.0,False,0.0,0,False,False,False,0.0,0.000000,0.0,0.0,0.0,0.0,TRACTION_BGK_OFF,2021-01-01 01:48:02.819180032,OCCCMS,OCC,1,TRACTION,TRACTION,"DM, DM, DI, DFS, DFN, DFN, DFS - Open Control",REQUESTED,Operational-Low,False,0.316497,True,0.643964,False,0.837822,False,0.772957,False,0.746246,0.663497,False,1
1,0.0,2021-01-01 01:50:14.752684032,1,OCCCMS,TRACTION_BGK_OFF,TERMINATED,NaN,"DM, DM, DI, DFS, DFN, DFN, DFS - Open Control",TRACTION,TRACTION,False,True,False,True,True,OCC,10,Operational-Low,3,False,4,False,1,True,0.0,0.0,0.0,0.3,0.3,0.994118,"1 - DM, DM, DI, DFS, DFN, DFN, DFS - Open Cont...",0.0,0,0,True,False,False,False,0.0,0,0.0,False,0.0,3,False,True,False,0.0,2.117271,0.0,0.0,0.0,0.0,TRACTION_BGK_OFF,2021-01-01 01:50:14.752684032,OCCCMS,OCC,1,TRACTION,TRACTION,"DM, DM, DI, DFS, DFN, DFN, DFS - Open Control",TERMINATED,Operational-Low,False,0.309214,False,0.362036,False,0.776042,False,0.520069,False,0.031734,0.399819,False,0
2,0.0,2021-01-02 00:37:56.958477056,1,OCCCMS,TRACTION_BGK_OFF,REQUESTED,NaN,"DM, DM, DI, DFS, DFN, DFN, DFS - Open Control",TRACTION,TRACTION,False,True,False,True,True,OCC,10,Operational-Low,3,False,5,True,0,True,0.0,0.0,0.0,0.3,0.3,0.313333,"1 - DM, DM, DI, DFS, DFN, DFN, DFS - Open Cont...",0.0,0,0,True,False,False,False,0.0,0,0.0,False,0.0,3,False,True,False,0.0,4.914142,0.0,0.0,0.0,0.0,TRACTION_BGK_OFF,2021-01-02 00:37:56.958477056,OCCCMS,OCC,1,TRACTION,TRACTION,"DM, DM, DI, DFS, DFN, DFN, DFS - Open Control",REQUESTED,Operational-Low,False,0.318976,False,0.425982,False,0.776042,False,0.550849,False,0.031734,0.420717,False,0
3,0.0,2021-01-02 00:40:23.753911040,1,OCCCMS,TRACTION_BGK_OFF,TERMINATED,NaN,"DM, DM, DI, DFS, DFN, DFN, DFS - Open Control",TRACTION,TRACTION,False,True,False,True,True,OCC,10,Operational-Low,3,False,5,True,0,True,0.0,0.0,0.0,0.3,0.3,0.310000,"1 - DM, DM, DI, DFS, DFN, DFN, DFS - Open Cont...",0.0,0,0,True,False,False,False,0.0,0,0.0,False,0.0,3,False,True,False,0.0,2.164353,0.0,0.0,0.0,0.0,TRACTION_BGK_OFF,2021-01-02 00:40:23.753911040,OCCCMS,OCC,1,TRACTION,TRACTION,"DM, DM, DI, DFS, DFN, DFN, DFS - Open Control",TERMINATED,Operational-Low,False,0.311145,False,0.359097,False,0.776042,False,0.534248,False,0.031734,0.402453,False,0
4,0.0,2021-01-03 00:52:01.285080064,1,OCCCMS,TRACTION_BGK_OFF,REQUESTED,NaN,"DM, DM, DI, DFS, DFN, DFN, DFS - Open Control",TRACTION,TRACTION,False,True,False,True,True,OCC,10,Operational-Low,3,False,6,True,0,True,0.0,0.0,0.0,0.3,0.3,0.263636,"1 - DM, DM, DI, DFS, DFN

In [27]:
# Inspect data
df.tail()

,EQUIPMENT_NAME,SCS_TIME,FUNCTIONAL_CATEGORY,ENVIRONMENT,ASSET_ID_RAW,EVENT_STATUS,ASSET_DESC_CAT,EVENT_DESC_CAT,AssetClass,AssetSubClass,isAlarm,NuisanceAlarm,RepeatAlarm,AltAlarm2,AltAlarm3,GeoCode,GeoSector,Severity_Class,SeverityRank,CrashWarning,DayofWeek,Weekend,HourofDay,EngHours,RWEC_Ratio_Flat,RWEC_Ratio_Seasonal,RWEC_Ratio_SeasonalAlarm,RWSS_LocAsset,RWSS_LocAssetClass,RWSS_Loc,EventAttr,sentimentScore,SentimentClass,SEVERITY_N-1,SEVERITY_Worsen,isAlarm_N-1,NuisanceAlarm_N-1,CrashWarning_N-1,sentimentScore_N-1,SentimentClass_N-1,sentimentScoreDelta,SentimentClassImprove_N-1,SCS_TIME_logDelta_N-1,SEVERITY_GEO_N-1,isAlarm_GEO_N-1,NuisanceAlarm_GEO_N-1,CrashWarning_GEO_N-1,sentimentScore_GEO_N-1,SCS_TIME_logDelta_GEO_N-1,RWEC_Ratio_Flat_GEO_N-1,RWEC_Ratio_Seasonal_GEO_N-1,RWEC_Ratio_SeasonalAlarm_GEO_N-1,EQUIPMENT_NAME,ASSET_ID_RAW,SCS_TIME,ENVIRONMENT,GeoCode,FUNCTIONAL_CATEGORY,AssetClass,AssetSubClass,EVENT_DESC_CAT,EVENT_STATUS,Severity_Class,Outlier_PCA,PCA_AnomalyProb,Outlier_IsoForest,IsoForest_AnomalyProb,Outlier_HBOS,HBOS_AnomalyProb,Outlier_CBLOF,CBLOF_AnomalyProb,Outlier_LODA,LODA_AnomalyProb,AnomalyProb,AnomalyProbClass,Outlier_Strength
312155,0.0,2021-01-03 01:46:21.702497024,74,OCCCMS,SCS/NED/1212/GWS04,SUCCEEDED,NelVisu,Operator Logged In/Out of NelVisu,SCS,GWS,False,False,False,False,False,OCC,10,Operational-Low,3,False,6,True,1,True,0.0,0.0,0.0,1.35,1.582143,1.354000,74 - Operator Logged In/Out of NelVisu - SUCCE...,0.0,0,0,True,False,False,False,0.0,0,0.0,False,0.0,3,False,False,False,0.0,0.602060,0.0,0.0,0.0,0.0,SCS/NED/1212/GWS04,2021-01-03 01:46:21.702497024,OCCCMS,OCC,74,SCS,GWS,Operator Logged In/Out of NelVisu,SUCCEEDED,Operational-Low,False,0.244243,False,0.280767,False,0.849186,False,0.147996,False,0.473629,0.399164,False,0
312156,0.0,2021-01-11 03:25:26.227955200,74,OCCCMS,SCS/NED/1212/GWS04,SUCCEEDED,NelVisu,Operator Logged In/Out of NelVisu,SCS,GWS,False,False,False,False,False,OCC,10,Operational-Low,3,False,0,False,3,True,0.0,0.0,0.0,1.35,1.905882,1.403571,74 - Operator Logged In/Out of NelVisu - SUCCE...,0.0,0,0,True,False,False,False,0.0,0,0.0,False,0.0,3,False,False,False,0.0,3.774079,0.0,0.0,0.0,0.0,SCS/NED/1212/GWS04,2021-01-11 03:25:26.227955200,OCCCMS,OCC,74,SCS,GWS,Operator Logged In/Out of NelVisu,SUCCEEDED,Operational-Low,False,0.249256,False,0.320808,False,0.849186,False,0.150845,False,0.473629,0.408745,False,0
312157,0.0,2021-01-11 03:25:26.734913024,74,OCCCMS,SCS/NED/1212/GWS04,SUCCEEDED,NelVisu,Operator Logged In/Out of NelVisu,SCS,GWS,False,False,False,False,False,OCC,10,Operational-Low,3,False,0,False,3,True,0.0,0.0,0.0,1.35,1.875000,1.401724,74 - Operator Logged In/Out of NelVisu - SUCCE...,0.0,0,0,True,False,False,False,0.0,0,0.0,False,0.0,3,False,False,False,0.0,0.000000,0.0,0.0,0.0,0.0,SCS/NED/1212/GWS04,2021-01-11 03:25:26.734913024,OCCCMS,OCC,74,SCS,GWS,Operator Logged In/Out of NelVisu,SUCCEEDED,Operational-Low,False,0.244247,False,0.265074,False,0.869366,False,0.121199,False,0.774272,0.454831,False,0
312158,0.0,2021-01-11 03:25:27.347742976,74,OCCCMS,SCS/NED/1212/GWS04,SUCCEEDED,NelVisu,Operator Logged In/Out of NelVisu,SCS,GWS,False,False,False,False,False,OCC,10,Operational-Low,3,False,0,False,3,True,0.0,0.0,0.0,1.35,1.847368,1.400000,74 - Operator Logged In/Out of NelVisu - SUCCE...,0.0,0,0,True,False,False,False,0.0,0,0.0,False,0.0,3,False,False,False,0.0,0.000000,0.0,0.0,0.0,0.0,SCS/NED/1212/GWS04,2021-01-11 03:25:27.347742976,OCCCMS,OCC,74,SCS,GWS,Operator Logged In/Out of NelVisu,SUCCEEDED,Operational-Low,False,0.244183,False,0.265301,False,0.869366,False,0.121003,False,0.774272,0.454825,False,0
312159,0.0,2021-01-13 01:19:49.153079040,74,OCCCMS,SCS/NED/1212/GWS04,SUCCEEDED,NelVisu,Operator Logged In/Out of NelVisu,SCS,GWS,False,False,False,False,False,OCC,10,Operational-Low,3,False,2,False,1,True,0.0,0.0,0.0,1.35,1.350000,1.350000,74 - Operator Logged In/Out of NelVisu - SUCCE...,0.0,0,0,True,False,False,False,0.0,0,0.0,False,0.0,3,False,False,False,0.0,4.896

In [28]:
# Drop Redundant Columns (Inspect data before dropping)
# Drop duplicate columns from df_anomaly to drop
df = df.loc[:, ~df.columns.duplicated()]

# Inspect data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312160 entries, 0 to 312159
Data columns (total 65 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   EQUIPMENT_NAME                    312160 non-null  float64
 1   SCS_TIME                          312160 non-null  object 
 2   FUNCTIONAL_CATEGORY               312160 non-null  int64  
 3   ENVIRONMENT                       312160 non-null  object 
 4   ASSET_ID_RAW                      312160 non-null  object 
 5   EVENT_STATUS                      312157 non-null  object 
 6   ASSET_DESC_CAT                    311610 non-null  object 
 7   EVENT_DESC_CAT                    312160 non-null  object 
 8   AssetClass                        312160 non-null  object 
 9   AssetSubClass                     307757 non-null  object 
 10  isAlarm                           312160 non-null  bool   
 11  NuisanceAlarm                     312160 non-null  b

In [29]:
# Delete redundant data
del df_eventAttr, df_anomaly

In [30]:
# Inspect Data
print("Anomalies by Ensemble Voting (Outlier Strength of 4 and above)")
print("Number of Anomalies before filtering: " + str(df[(df["Outlier_Strength"] >= 4)].shape[0]))
#print("Number of Anomalies after filtering: " + str(df[(df["Outlier_Strength"] >= 4) & (df["Benign Event To Ignore"] >= False)].shape[0]))

Anomalies by Ensemble Voting (Outlier Strength of 4 and above)
Number of Anomalies before filtering: 1


In [31]:
# Inspect Data
print("Anomalies by Mean Anomaly Probality (Mean Probability of 70% and above)")
print("Number of Anomalies before filtering: " + str(df[(df["AnomalyProbClass"] == True)].shape[0]))
#print("Number of Anomalies after filtering: " + str(df[(df["AnomalyProbClass"] == True) & (df["Benign Event To Ignore"] >= False)].shape[0]))

Anomalies by Mean Anomaly Probality (Mean Probability of 70% and above)
Number of Anomalies before filtering: 4


## Export File

In [32]:
# Check current directory
cwd

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs'

In [33]:
# Define Save Location
saveLoc = saveLoc
#os.chdir(cwd + alarmLoc)
os.chdir(cwd + saveLoc)
# Check directory location
print(os.getcwd())

C:\Users\cftfda01\Documents\SBST Train IAMS Project\alarm-event-logs\taggedOutput\AnomalyTaggingResultsFULL\run 05\cms


In [34]:
# Define File Save Parameters
FileName = targetFiles
Run = "-B0001"
# True for single file output; # False for multiple file output; "both" for both Single & Multiple File Output
singleSave = "both"
# Define source data
df = df

# Get length of dataframe
df_len = len(df)
# Inspect data
print(df_len)

# Define Size of Partitioned Dataframes
partionSize = 500000

# Define Number of Partitions (Always Round Up to Nearest Interger)
if (df_len == partionSize): 
    partitions = 1
else:
    partitions = df_len // partionSize + 1

# Inspect data
print(partitions)


312160
1


In [35]:
# Export file based on above settings
if singleSave == True:
    fileNameN = FileName + EngHrTag + Run + '-anomalyTaggedFULL' + '.csv'
    df.to_csv("main/" + fileNameN, index=False)
    print(fileNameN + " SAVED")
elif singleSave == "both":
    # Single File Save
    fileNameN = FileName + EngHrTag + Run + '-anomalyTaggedFULL' + '.csv'
    df.to_csv("main/" + fileNameN, index=False)
    print(fileNameN + " SAVED")
    
    # Split Dataframe into batches of 500K rows
    for counter in range(partitions):
        startPoint = counter * partionSize
        df_subset = df.iloc[startPoint : (startPoint + partionSize)]
        #print(df_subset.info())

        # Save File
        fileNameN = "Subset File Ver/" + FileName + EngHrTag +  Run + '-' + str(counter).zfill(3) + '-anomalyTaggedFULL' + '.csv'
        df_subset.to_csv(fileNameN, index=False)
        print(fileNameN + " SAVED")
else:
    # Split Dataframe into batches of 500K rows
    for counter in range(partitions):
        startPoint = counter * partionSize
        df_subset = df.iloc[startPoint : (startPoint + partionSize)]
        #print(df_subset.info())

        # Save File
        fileNameN = "Subset File Ver/" + FileName + EngHrTag +  Run + '-' + str(counter).zfill(3) + '-anomalyTaggedFULL' + '.csv'
        df_subset.to_csv(fileNameN, index=False)
        print(fileNameN + " SAVED")
        
# Ring Beeper When Complete
import winsound
import time

duration1 = 400  # milliseconds
freq1 = 400  # Hz
duration2 = 600  # milliseconds
freq2 = 300  # Hz
repeatCount = 7
for n in range(repeatCount):
    winsound.Beep(freq1, duration1)
    winsound.Beep(freq2, duration2)
    time.sleep(1)

cms-EngHr-B0001-anomalyTaggedFULL.csv SAVED
Subset File Ver/cms-EngHr-B0001-000-anomalyTaggedFULL.csv SAVED
